In [2]:
import numpy as np
from bisect import bisect_right as BSR, bisect_left as BSL 

def generateSymetryBase():
    s3d2 = np.sqrt(3)/2
    return {
        "c":{
            "000":np.array([[-1,0,0],[0,-1,0],[0,0,-1]])
        },
        "m":{
            "100":np.array([[-1,0,0],[0,1,0],[0,0,1]]),
            "010":np.array([[1,0,0],[0,-1,0],[0,0,1]]),
            "001":np.array([[1,0,0],[0,1,0],[0,0,-1]]),

            "110":np.array([[0,-1,0],[-1,0,0],[0,0,1]]),            
            "011":np.array([[1,0,0],[0,0,-1],[0,-1,0]]),
            "101":np.array([[0,0,-1],[0,1,0],[-1,0,0]]),
            
            "-101":np.array([[0,0,1],[0,1,0],[1,0,0]]),
            "-110":np.array([[0,1,0],[1,0,0],[0,0,1]]),
            "0-11":np.array([[1,0,0],[0,0,1],[0,1,0]])
        },
        "2":{
            "100":np.array([[1,0,0],[0,-1,0],[0,0,-1]]),
            "010":np.array([[-1,0,0],[0,1,0],[0,0,-1]]),
            "001":np.array([[-1,0,0],[0,-1,0],[0,0,1]]),

            "110":np.array([[0,1,0],[1,0,0],[0,0,-1]]),
            "101":np.array([[1,0,0],[0,-1,0],[0,0,1]]),
            "011":np.array([[-1,0,0],[0,0,1],[0,1,0]]),

            "-110":np.array([[0,-1,0],[-1,0,0],[0,0,-1]]),
            "-101":np.array([[0,0,-1],[0,-1,0],[-1,0,0]]),
            "0-11":np.array([[-1,0,0],[0,0,-1],[0,-1,0]])
        },
        "3":{
#             "001":np.array([[-0.5,-s3d2,0],[s3d2,-0.5,0],[0,0,1]]),
            "111":np.array([[0,0,1],[1,0,0],[0,1,0]]),
            "-111":np.array([[0,-1,0],[0,0,1],[-1,0,0]]),
            "1-11":np.array([[0,-1,0],[0,0,-1],[1,0,0]]),
            "11-1":np.array([[0,1,0],[0,0,-1],[-1,0,0]])
        },
        "-3":{
#             "001":np.array([[0.5,s3d2,0],[-s3d2,0.5,0],[0,0,-1]]),
            "111":np.array([[0,0,-1],[-1,0,0],[0,-1,0]]),
            "-111":np.array([[0,1,0],[0,0,-1],[1,0,0]]),
            "1-11":np.array([[0,1,0],[0,0,1],[-1,0,0]]),
            "11-1":np.array([[0,-1,0],[0,0,1],[1,0,0]])        
        },
        "4":{
            "100":np.array([[1,0,0],[0,0,-1],[0,1,0]]),
            "010":np.array([[0,0,1],[0,1,0],[-1,0,0]]),
            "001":np.array([[0,-1,0],[1,0,0],[0,0,1]])
        },
        "-4":{
            "100":np.array([[-1,0,0],[0,0,1],[0,-1,0]]),
            "010":np.array([[0,0,-1],[0,-1,0],[1,0,0]]),
            "001":np.array([[0,1,0],[-1,0,0],[0,0,-1]])
        }#,
#         "6":{
#             "001":np.array([[0.5,-s3d2,0],[s3d2,0.5,0],[0,0,1]])
#         },
#         "-6":{
#             "001":np.array([[-0.5,s3d2,0],[-s3d2,-0.5,0],[0,0,-1]])
#         }
    }

In [3]:
# def listadous(macierz, punktprzes):
#     punkt = np.matmul(macierz,punktprzes)  # np.matmul
#     punkt = np.around(punkt,6)
#     while not np.allclose(punkt, punktprzes):   # porownajPunkty
#         yield punkt
#         punkt = np.matmul(macierz,punkt)   
#         punkt = np.around(punkt,6) 
        
def listadous(macierz, punktprzes):
    punkt = np.matmul(macierz,punktprzes)
    while not porownajPunkty(punkt, punktprzes):
        yield punkt
        punkt = np.matmul(macierz,punkt)        
        
# def findAntiPoints(mylist,zbior2):
#     for punktprzek in mylist:
#         indx = findindex(punktprzek,zbior2)               
#         if indx + 1 and porownajPunkty(zbior2[:,indx],punktprzek): #porownajPunkty()
#             return False
#     return True    

"""def supercell(cellorg, size):
    if size == 1:
        return np.unique(cellorg,axis=0)*2-1
    elif size < 1:
        return None
    cell = cellorg.copy()
    for os in range(3):    
        subcell = cell.copy()[cell.T[os]!=0]
        listcell = subcell.copy()
        for _ in range(1,size):        
            subcell[:,os] += 1
            listcell = np.append(listcell,subcell,axis =0)   
        cell = np.append(cell,listcell,axis =0)   
    cell = np.unique(cell,axis = 0)/(size/2)-1
    return np.around(cell,6)"""

def myswitch(nr):
    mydict = {
        "c": 1,
        "m": 1,
        "2": 1,
        "3": 2,
        "-3": 5,
        "4": 3,
        "-4": 3,
        "6": 5,
        "-6": 5
    }
    return mydict[nr]

# def makeCellWithVacancies(cell,indexes):
#     indexes = set(indexes)
#     cellVac = []
#     Vac = []
#     for nr, p in enumerate(cell):
#         if nr not in indexes:
#             cellVac.append(p)
#         else:
#             Vac.append(p)
#     return np.array(cellVac), np.array(Vac)

'''def listadous(macierz, punktprzes):
    punkt = np.matmul(macierz,punktprzes)  # np.matmul
    punkt = np.around(punkt,6)
    while not porownajPunkty(punkt, punktprzes):   # porownajPunkty
        yield punkt
        punkt = np.matmul(macierz,punkt)   
        punkt = np.around(punkt,6) '''

def usunkoor(koorZEW, oski):
    koorWEW = koorZEW.copy()
    for os in oski:
        j = 1
        while j < len(koorWEW):            
            for punktdous in listadous(os, koorWEW[j]):                
                i = findindex(punktdous, koorWEW.T[:,j:])                
                if i + 1 and porownajPunkty(koorWEW[i+j],punktdous):
                    koorWEW = np.delete(koorWEW, i+j, 0)
            j += 1
    return koorWEW

def usunkoor_mod(koorZEW, oski):#, Matrixes):
    koorWEW = koorZEW.copy()
    listofnr = list(range(len(koorWEW)))
    Matrixes = generateSymetryBase()
    for os0, os1 in oski:
        TRANS = Matrixes[os0][os1]
        j = 1
        while j < len(koorWEW):            
            for punktdous in listadous(TRANS, koorWEW[j]):                
                i = findindex(punktdous, koorWEW.T[:,j:])                
                if i + 1 and porownajPunkty(koorWEW[i+j],punktdous):
                    koorWEW = np.delete(koorWEW, i+j, 0)
                    del listofnr[i+j]
            j += 1
    return listofnr#zip(koorWEW, listofnr)

def makelist():
    Matrixes = generateSymetryBase()
    mylist = []
    for Mainkey in Matrixes.keys():
        for Subkey in Matrixes[Mainkey].keys():
            mylist.append((Mainkey,Subkey))
    return mylist[::-1]

def porownajPunkty(p1,p2):
    for a, b in zip(p1,p2):
        if a!=b:
            return False
    return True

def porownajMacierze(m1,m2):
    for a, b in zip(m1,m2):
        if not porownajPunkty(a,b):
            return False
    return True

def findindex(searched, points):         
    indexL, indexR = 0, len(points[0])        
    for xyz in range(len(points)): 
        numR = BSR(points[xyz][indexL: indexR], searched[xyz])
        numL = BSL(points[xyz][indexL: indexR], searched[xyz])        
        if numL < numR:
            indexR = indexL + numR
            indexL += numL
        elif numL == numR:
            indexL += numL
            if indexL != len(points[0]):
                return indexL
            return -1
        else:
            return -1        
    return indexL 

# def findAntiPoints(listofvac, allpoints2):
#     for vacsym in listofvac:
#         indx = findindex(vacsym,allpoints2)               
#         if indx + 1 and porownajPunkty(allpoints2[:,indx],vacsym): #porownajPunkty()
#             return False
#     return True

def findPoints_MIXED(listofps, allpoints2, Anti = False):
    for ps in listofps:
        indx = findindex(ps,allpoints2)  
        cond = indx + 1 and porownajPunkty(allpoints2[:,indx],ps) #porownajPunkty() 
        if Anti == cond:           
                return False
    return True 

def findAntiSym_InnerLoop(Matrix, allpoints, vacancies):
    for vac in vacancies:
        listofvacsym = listadous(Matrix,vac)
        if not findPoints_MIXED(listofvacsym, allpoints.T, Anti=True):
            return False
    return True

def findAntiSym_MOD(matrixes, allpoints, vacancies):
    allsymmerties = makelist()
    if vacancies.shape == (3,):
        vacancies = np.array([vacancies])
    possymmerties = []    
    for el0, el1 in allsymmerties:
        if findAntiSym_InnerLoop(matrixes[el0][el1], allpoints, vacancies):
            possymmerties.append((el0, el1))           
    return possymmerties 

def findSym_Base_mod2_innerLoop(Matrix, allpoints): #to samo co w anty?
    for p in allpoints:
        listofps = listadous(Matrix,p)          
        if not findPoints_MIXED(listofps,allpoints.T):
            return False
    return True

def findSym_Base_mod2(matrixes, allpoints, vacancies):
    symmerties = []
    possym = findAntiSym_MOD(matrixes, allpoints, vacancies)
    for el0, el1 in possym:                
        if findSym_Base_mod2_innerLoop(matrixes[el0][el1], allpoints):
            symmerties.append((el0, el1))
    return symmerties

In [4]:
def poprownp(oldarr,i):
    pop = oldarr[i]
    newarr = np.vstack((oldarr[:i],oldarr[i+1:]))
    return newarr, pop

def makeCellWithVacancies(cell,indexes):
    indexes = set(indexes)
    cellVac = []
    Vac = []
    for nr, p in enumerate(cell):
        if nr not in indexes:
            cellVac.append(p)
        else:
            Vac.append(p)
    return np.array(cellVac), np.array(Vac)

# print(makeCellWithVacancies(SUPERCELL,(0,)))
# print(poprownp(SUPERCELL,0))

In [5]:
# def qwerty(numerki1,numerki2):
#     uvw = numerki1-numerki2
#     if np.any(uvw > 1) or np.any(uvw < -1):
# #         print(True)
#         return (numerki1,numerki2)
# #     else:
# #         print(False)
    
# # %timeit qwerty(np.array([-1,0,1]),np.array([-1,-1,-1]))

def makeUVW(num1,num2):
    uvw = np.abs(num1-num2)
    return uvw
#     if np.any(uvw > 1):
#         return True
#     return False

# %timeit qwerty2(np.array([-1,0,1]),np.array([-1,-1,-1]))

def powtorka(UVW):
    myset = set(UVW)
    if len(myset) == 2:
        return False
    return True

def sdgk(UVW):
    mystr = ''
    for el in UVW:
        mystr += str(el)
    return mystr



def myfunc(num1, num2):
    UVW = np.abs(num1-num2)
    UVW = UVW/np.amax(UVW)
    return porownajPunkty(UVW,UVW.astype(int))

def myfunc2(num1, num2):
    mylist = []
    mylist.append(("num1",num1))
    mylist.append(("num2",num2))
    UVW = np.abs(num1-num2)
    mylist.append(("make  UVW",UVW))
    UVW = UVW/np.amax(UVW)
    mylist.append(("UVW /amax",UVW))   
    mylist.append(('type  int',UVW.astype(int)))
    val = porownajPunkty(UVW,UVW.astype(int))
    if val:
        print('-----------------------------------------------------------------------------------------------------------------')
    mylist.append(val)
    mylist.append(("num1-num2",num1-num2))
    mylist.append(("num1+num2",num1+num2))
    mylist.append(("num1*num2",num1*num2))
    mylist.append(("num1/num2",num1/num2))
    mylist.append(("num1%num2",num1%num2))
    mylist.append(("num2%num1",num2%num1))
    mylist.append(("matmul 12",np.matmul(num1,num2)))
    
    return mylist
    
num1 = np.array([-1,1,0.5])
num2 = np.array([0,1,0.5])
# print(myfunc2(num1,num2))
# print(makeUVW(num1,num2))
# %timeit myfunc2(num1,num2)

In [6]:
# p = np.array([0.5,-1,0])
# print(np.matmul(np.array([[0,1,0],[0,0,1],[-1,0,0]]),p))
# print(np.array([p[1],p[2],-p[0]])) 

In [7]:
# %timeit makeCellWithVacancies(SUPERCELL,(0,))
# %timeit poprownp(SUPERCELL,0)

In [8]:
def abcde(scell,sumVac = 1):
    mylist = []
    for indexes in itrtls_combinations(range(len(scell)),sumVac): 
#         print(indexes)
        scellvac, vacancies = makeCellWithVacancies(scell,indexes)
        SYM = findSym_Base_mod2(Matrixes, scellvac, vacancies)
        mylist.append(SYM)
    return mylist

# jakaslista = usunkoor_mod(SUPERCELL, oski)
# def abcdef(scell, sumVac, uskoor):
#     mylist = []
#     for index in uskoor: 
#         scellvac, vacancies = poprownp(scell, index)
#         mozliwosci = findAntiSym_MOD(Matrixes, scellvac, vacancies)
#         SYM = findSym_Base_mod2(Matrixes, scellvac, mozliwosci)
#         if sumVac:
#             uskoor2 = usunkoor_mod(SUPERCELL, SYM)
#             listaextend.s. abcdef(scellvac, sumVac - 1, uskoor2)
#         else:
#             mylist.append(SYM)
#     return mylist
# #przesłac jakies jot minimalne 

def abcd(scell,sumVac):
    mylist = []
    count = 0 
    count2_1 = 0 
#     count2_2 = 0
    countcond = 0 
    countall = 0
    for indexes in itrtls_combinations(range(len(scell)),sumVac):   
#         print(indexes)
        scellvac, vacancies = makeCellWithVacancies(scell,indexes)
        SYM = findSym_Base_mod2(Matrixes, scellvac, vacancies)
#         mylist.append(SYM)     
        cond1 = SYM == []
        cond2 = myfunc(*vacancies) 
        if cond1:
            count += 1   
            for el in myfunc2(*vacancies):
                print(el)
            print()
        if cond2:
            count2_1 += 1          
#         else:
#             count2_2 += 1
        if cond2 and cond1:
            countcond += 1           
#             print()
        countall += 1
#         if count > 10:
#             break
    print("ile wszystkiego ",countall)
    print("ile sym = []  ",count)
    print("ile UVW true ",count2_1)
    print("ile UVW false ",countall - count2_1)
    print("oba takie same war ",countcond)
    return mylist

In [9]:
def string2array(direction):
    lenght = len(direction.replace("-",''))
    arr = np.empty(lenght)
    j = False
    i = 0
    k = 0 
    while i < len(direction): 
        char = direction[i]
        if char == "-":
            j = True
            i += 1
            k += 1
            continue
        if j:
            arr[i-k] = -int(char)
            j = False
        else:
            arr[i-k] = int(char)
        i += 1 
    return arr
%timeit string2array("001")

3.19 µs ± 28.1 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [10]:
def isPerpendicular(UVW1, UVW2):
    return not np.sum(UVW1*UVW2) 
UVW1 = np.array([1,0,1])
UVW2 = np.array([1,0,-1])
# %timeit isPerpendicular(UVW1, UVW2)

In [11]:
# scellvac, vacancies = makeCellWithVacancies(SUPERCELL,(54,76))
# SYM = findSym_Base_mod2(Matrixes, scellvac, vacancies)
# print(SYM)

In [12]:
# for kier in SYM:
#     print(string2array(kier[1]))

In [13]:
from Profiling import profile
# from MMfunc import *
from cifParsing import *
# from crystals import Crystal
from itertools import combinations as itrtls_combinations

filename = 'ZnS-Sfaleryt.cif'
# filename = '9008064.cif'
SUPERCELL = getSCell(Crystal.from_cif, filename, size = 1)
print(len(SUPERCELL))
Matrixes = generateSymetryBase()

18


In [14]:
# print(SUPERCELL)

In [15]:
SUPERCELL2 = getSCell(Crystal.from_cif, filename, size = 2)
%time lol2 = abcde(SUPERCELL2,2) #SC:4 VAC:2 Wall time:24min 27s

Wall time: 8.63 s


In [9]:
for SIZE in range(1,3):
    for VAC in range(1,3):
        print(SIZE, VAC)
        SUPERCELL = getSCell(Crystal.from_cif, filename, size = SIZE)
        %time abcde(SUPERCELL,sumVac = VAC)
        print()

1 1
Wall time: 110 ms

1 2
Wall time: 666 ms

2 1
Wall time: 1.15 s

2 2
Wall time: 26.2 s



In [16]:
for el in sorted(lol2, key=len, reverse=True):
    if el != []:
        print(el)

[('-4', '100'), ('2', '001'), ('2', '010'), ('2', '100'), ('m', '0-11'), ('m', '011')]
[('-4', '010'), ('2', '001'), ('2', '010'), ('2', '100'), ('m', '-101'), ('m', '101')]
[('-4', '001'), ('2', '001'), ('2', '010'), ('2', '100'), ('m', '-110'), ('m', '110')]
[('3', '111'), ('m', '0-11'), ('m', '-110'), ('m', '-101')]
[('3', '111'), ('m', '0-11'), ('m', '-110'), ('m', '-101')]
[('3', '111'), ('m', '0-11'), ('m', '-110'), ('m', '-101')]
[('3', '111'), ('m', '0-11'), ('m', '-110'), ('m', '-101')]
[('3', '11-1'), ('m', '-110'), ('m', '101'), ('m', '011')]
[('3', '11-1'), ('m', '-110'), ('m', '101'), ('m', '011')]
[('3', '11-1'), ('m', '-110'), ('m', '101'), ('m', '011')]
[('3', '11-1'), ('m', '-110'), ('m', '101'), ('m', '011')]
[('3', '1-11'), ('m', '-101'), ('m', '011'), ('m', '110')]
[('3', '1-11'), ('m', '-101'), ('m', '011'), ('m', '110')]
[('3', '1-11'), ('m', '-101'), ('m', '011'), ('m', '110')]
[('3', '1-11'), ('m', '-101'), ('m', '011'), ('m', '110')]
[('3', '-111'), ('m', '0-11

In [21]:
print(len(lol2))
countempty = 0
for el in sorted(lol2, key=len, reverse=True):
    if el != []:
        print(el)
    else:
        countempty += 1

4465
(array([[-1., -1., -1.],
       [-1., -1.,  0.]]), [('m', '-110')])
(array([[-1., -1., -1.],
       [-1., -1.,  1.]]), [('m', '-110')])
(array([[-1. , -1. , -1. ],
       [-1. , -0.5, -0.5]]), [('m', '0-11')])
(array([[-1. , -1. , -1. ],
       [-1. , -0.5,  0.5]]), [])
(array([[-1., -1., -1.],
       [-1.,  0., -1.]]), [('m', '-101')])
(array([[-1., -1., -1.],
       [-1.,  0.,  0.]]), [('m', '0-11')])
(array([[-1., -1., -1.],
       [-1.,  0.,  1.]]), [])
(array([[-1. , -1. , -1. ],
       [-1. ,  0.5, -0.5]]), [])
(array([[-1. , -1. , -1. ],
       [-1. ,  0.5,  0.5]]), [('m', '0-11')])
(array([[-1., -1., -1.],
       [-1.,  1., -1.]]), [('m', '-101')])
(array([[-1., -1., -1.],
       [-1.,  1.,  0.]]), [])
(array([[-1., -1., -1.],
       [-1.,  1.,  1.]]), [('2', '100'), ('m', '0-11'), ('m', '011')])
(array([[-1.  , -1.  , -1.  ],
       [-0.75, -0.75, -0.75]]), [('3', '111'), ('m', '0-11'), ('m', '-110'), ('m', '-101')])
(array([[-1.  , -1.  , -1.  ],
       [-0.75, -0.75,  0

In [19]:
# from collections import Counter

In [ ]:
print(len(lol))
for el in sorted(lol):
    if not el == []:
        print(el)

In [10]:
print(len(lol2))
countempty = 0
for el in sorted(lol2):
    if el != []:
        print(el)
    else:
        countempty += 1

39060
[('-4', '001'), ('2', '001'), ('2', '010'), ('2', '100'), ('m', '-110'), ('m', '110')]
[('-4', '001'), ('2', '001'), ('2', '010'), ('2', '100'), ('m', '-110'), ('m', '110')]
[('-4', '010'), ('2', '001'), ('2', '010'), ('2', '100'), ('m', '-101'), ('m', '101')]
[('-4', '010'), ('2', '001'), ('2', '010'), ('2', '100'), ('m', '-101'), ('m', '101')]
[('-4', '100'), ('2', '001'), ('2', '010'), ('2', '100'), ('m', '0-11'), ('m', '011')]
[('-4', '100'), ('2', '001'), ('2', '010'), ('2', '100'), ('m', '0-11'), ('m', '011')]
[('2', '001')]
[('2', '001')]
[('2', '001')]
[('2', '001')]
[('2', '001')]
[('2', '001')]
[('2', '001')]
[('2', '001')]
[('2', '001')]
[('2', '001')]
[('2', '001')]
[('2', '001')]
[('2', '001')]
[('2', '001')]
[('2', '001')]
[('2', '001')]
[('2', '001')]
[('2', '001')]
[('2', '001')]
[('2', '001')]
[('2', '001')]
[('2', '001')]
[('2', '001')]
[('2', '001')]
[('2', '001')]
[('2', '001')]
[('2', '001')]
[('2', '001')]
[('2', '001')]
[('2', '001')]
[('2', '001')]
[('2', 

In [14]:
print(countempty)

0


In [21]:
%timeit usunkoor_mod(SUPERCELL, sorted(lol2)[0])

5.97 ms ± 37.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [5]:
%time %timeit -r 10 -n 100000 np.matmul(np.array([[0,1,0],[0,0,1],[-1,0,0]]),p)
%time %timeit -r 10 -n 100000 np.array([p[2],p[0],p[1]])

4.3 µs ± 40.6 ns per loop (mean ± std. dev. of 10 runs, 100000 loops each)
Wall time: 4.3 s
1.53 µs ± 11.3 ns per loop (mean ± std. dev. of 10 runs, 100000 loops each)
Wall time: 1.53 s
